In [1]:
import os, sys
import reaxff_convert as rc
import pandas as pd
import numpy as np

from reaxFF_data import reaxFF_data, bidict

## Originating Files

In [2]:
species = ['C', 'H', 'O', 'Fe']
potential_fd = os.path.join(os.getcwd(), "potentials")

water_fd = os.path.join(potential_fd, "ffield.reax.O_H")
fe_o_water_fd = os.path.join(potential_fd, "ffield.reax.Fe_O_C_H")
fe_c_h_fd = os.path.join(potential_fd, "ffield.reax.Fe_C_H")
fe_o_c_fd = os.path.join(potential_fd, "ffield.reax.Fe_O_C")

water_data = reaxFF_data(['H', 'O'])
water_data.read_lammps(water_fd)
water_data.clean_params()

fe_o_water_data = reaxFF_data(species)
fe_o_water_data.read_lammps(fe_o_water_fd)
fe_o_water_data.clean_params()

fe_c_h_data = reaxFF_data(['H', 'C', 'Fe'])
fe_c_h_data.read_lammps(fe_c_h_fd)
fe_c_h_data.clean_params()

fe_o_c_data = reaxFF_data(['C', 'O', 'Fe'])
fe_o_c_data.read_lammps(fe_o_c_fd)
fe_o_c_data.clean_params()

## Starting Compilation

In [3]:
master_data = reaxFF_data(species)
for i,k in master_data.params.items():
    k['source'] = float('nan')
steps = [master_data.copy()]

In [4]:
# First make the new df
steps.append(steps[-1].copy())

# # Combine data
for i, k in steps[-1].params.items():
    if i == 'general':
        k.update(fe_o_water_data.params[i])
        k['source'] = 'fe_o_water'
    else:
        cur_param_df = fe_o_water_data.params[i][k.columns.difference(['source'])]
        changes_df = k.reset_index().merge(cur_param_df.reset_index(),how="right").set_index('symbols')
        changes_df['source'] = 'fe_o_water'
        k.update(changes_df)

In [5]:
# Combine new OH data
# First make the new df
steps.append(steps[-1].copy())

# # Combine data
for i, k in steps[-1].params.items():
    if i == 'general':
        continue
    else:
        cur_param_df = water_data.params[i][k.columns.difference(['source'])]
        changes_df = k.reset_index().merge(cur_param_df.reset_index(),how="right").set_index('symbols')
        changes_df['source'] = 'water'
        k.update(changes_df)

## Different Merging with Fe/C/O

In [10]:
from itertools import combinations
par_list = []
par_combi = ['bonds', 'off_diagonal', 'angles']

# Inspection revealed no torsion changes
# par_list.extend(list(combinations(['bonds', 'off_diagonal', 'angles', 'torsions'], r=4)))
par_list.extend(list(combinations(par_combi, r=3)))
par_list.extend(list(combinations(par_combi, r=2)))
par_list.extend(list(combinations(par_combi, r=1)))
par_list

[('bonds', 'off_diagonal', 'angles'),
 ('bonds', 'off_diagonal'),
 ('bonds', 'angles'),
 ('off_diagonal', 'angles'),
 ('bonds',),
 ('off_diagonal',),
 ('angles',)]

In [11]:
import datetime
    
for par in par_list:
    # First make the new df
    new_step = steps[-1].copy()
    
    # # Combine data
    for i, k in new_step.params.items():
        if i not in par:
            continue
        else:
            cur_param_df = fe_o_c_data.params[i][k.columns.difference(['source'])]
            cur_param_df = cur_param_df[[('C' in x and 'Fe' in x) for x in cur_param_df.index]]
            changes_df = k.reset_index().merge(cur_param_df.reset_index(),how="right").set_index('symbols')
            changes_df['source'] = 'fe_o_c'
            print(' '.join(par))
            print("*"*30)
            print(changes_df)
            print("*"*30)
            k.update(changes_df)

    new_step.description = "Combined fe_o_water with new water and fe_o_c's "+ '-'.join(par) + str(datetime.datetime.now())
    new_step.clean_params()
    new_step.gen_species2id()
    new_step_str = new_step.write_lammps()

    with open(os.path.join("final_potential_set", '_'.join(par) + "_c_fe.reax"), 'w') as f:
        f.write(new_step_str)

bonds off_diagonal angles
******************************
           Edis1 LPpen n.u.1    pbe1 pbo5 13corr pbo6   kov    pbe2  pbo3  \
symbols                                                                     
C-Fe     78.6009     0     0  0.8712 -0.3      1   36  0.01  1.1811 -0.35   

        pbo4 Etrip    pbo1   pbo2 ovcorr n.u.2  source  
symbols                                                 
C-Fe      15     1 -0.1666  2.611      1     0  fe_o_c  
******************************
bonds off_diagonal angles
******************************
          Ediss      Ro  gamma  rsigma rpi rpi2   source
symbols                                                 
C-Fe     0.5077  0.3058  6.291  1.2466  -1    -1  fe_o_c
******************************
bonds off_diagonal angles
******************************
         Theta,o       ka       kb pv1     pv2 kpenal     pv3  source
symbols                                                              
C-C-Fe     7.031  17.9932   0.4327   0  0.1623      0

In [12]:
import datetime

# First make the new df
new_step = steps[-1].copy()

# # Combine data
for i, k in new_step.params.items():
    if i not in ['angles']:
        continue
    else:
        cur_param_df = fe_o_c_data.params[i][k.columns.difference(['source'])]
        cur_param_df = cur_param_df[[('C' in x and 'Fe' in x and 'O' in x) for x in cur_param_df.index]]
        changes_df = k.reset_index().merge(cur_param_df.reset_index(),how="right").set_index('symbols')
        changes_df['source'] = 'fe_o_c'
        k.update(changes_df)

new_step.description = "Combined fe_o_water with new water and fe_o_c's Fe C and O"+ '-'.join(par) + str(datetime.datetime.now())
new_step.clean_params()
new_step.gen_species2id()
new_step_str = new_step.write_lammps()

with open(os.path.join("final_potential_set", 'angles' + "_o_c_fe.reax"), 'w') as f:
    f.write(new_step_str)